### Importaciones

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from time import time

# --- Módulos de Scikit-learn ---
from sklearn.svm import LinearSVC  # ¡Tu modelo! (Más rápido que SVC para texto)
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score

# --- Módulos propios (¡Los ganadores!) ---
# 1. Normalización Ganadora
from normalization_functions import pipeline_d_normalize
# 2. Representación Ganadora (Binary + Lexicons)
from lexicon_functions import get_augmented_features

# --- Configuración ---
warnings.filterwarnings('ignore')
print("¡Librerías de Fase 3 (SVM) cargadas!")

Recurso 'stopwords' de NLTK ya está descargado.
Cargadas 313 stopwords en español.
Modelo 'es_core_news_sm' de spaCy cargado.
¡Librerías de Fase 3 (SVM) cargadas!


### Carga y PRE-Normalización de Datos

In [ ]:
# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
# ¡Tu archivo de resultados!
RESULTS_PATH = os.path.join("..", "results", "fase_3_svm_results.csv")

os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)

# --- 2. Cargar Datos ---
print("Iniciando Fase 3: SVM + GridSearchCV")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    df_train.dropna(subset=['text', 'Polarity'], inplace=True)
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {TRAIN_PATH}")
    # Si da error, detén la ejecución (en un notebook, esto es simbólico)
    raise

# --- 3. Aplicar Normalización GANADORA (Paso 1) ---
print(f"Aplicando 'pipeline_d_normalize' a {len(df_train)} opiniones...")
start_norm = time()

X_train_normalized = df_train['text'].apply(pipeline_d_normalize)
y_train = df_train['Polarity']

end_norm = time()
print(f"Datos normalizados listos. (Tiempo: {end_norm - start_norm:.2f}s)")

Iniciando Fase 3: SVM + GridSearchCV
Aplicando 'pipeline_d_normalize' a 24169 opiniones...
Datos normalizados listos. (Tiempo: 2.36s)


### Definir Pipeline y Rejilla de Búsqueda

In [3]:
# --- 4. Definir Pipeline de Modelo ---
# Pipeline:
# 1. 'features': La representación ganadora (Binary + Lexicons)
# 2. 'classifier': Tu modelo SVM

svm_pipeline = Pipeline([
    ('features', get_augmented_features()), #
    ('classifier', LinearSVC(random_state=42, max_iter=3000)) # Modelo
])

# --- 5. Definir Rejilla de Hiperparámetros ---
# 'classifier__C' se refiere al parámetro 'C' del paso 'classifier' (LinearSVC)
# 'C' es el parámetro de regularización de SVM.
# Valores bajos (0.1) -> Más regularización (modelo más simple)
# Valores altos (10) -> Menos regularización (modelo más complejo)
param_grid = {
    'classifier__C': [0.1, 1, 10]
}

# --- 6. Configurar GridSearchCV ---
# verbose=3 para que se vea el progreso
grid_search = GridSearchCV(
    estimator=svm_pipeline,
    param_grid=param_grid,
    cv=5,                # 5 folds
    scoring='f1_macro',  # Métrica
    n_jobs=-1,           # Usar todos los núcleos de CPU
    verbose=3            
)

print("GridSearchCV (SVM) configurado.")
print(f"Probando {len(param_grid['classifier__C'])} valores de 'C' con 5-fold CV...")
print("Total de entrenamientos: ", len(param_grid['classifier__C']) * 5)

GridSearchCV (SVM) configurado.
Probando 3 valores de 'C' con 5-fold CV...
Total de entrenamientos:  15


In [4]:
# --- 7. Ejecutar GridSearchCV ---

print("--- ¡INICIANDO GRIDSEARCHCV (SVM)! ---")
print(f"Alimentando el modelo con {len(X_train_normalized)} opiniones normalizadas.")
print("Esto puede tardar varios minutos (o más)... ten paciencia.")

start_grid = time()

try:
    # Aquí es donde se ejecuta el trabajo pesado:
    # Probará C=0.1 (con 5 folds), C=1 (con 5 folds) y C=10 (con 5 folds)
    grid_search.fit(X_train_normalized, y_train)
    
except Exception as e:
    print(f"\n¡Error durante el entrenamiento de GridSearchCV! {e}")
    # Si hay un error, lo saltamos para poder seguir
    pass

end_grid = time()
print(f"\n--- ¡GridSearchCV (SVM) completado! ---")
print(f"Tiempo total de búsqueda: {(end_grid - start_grid) / 60:.2f} minutos")

--- ¡INICIANDO GRIDSEARCHCV (SVM)! ---
Alimentando el modelo con 24169 opiniones normalizadas.
Esto puede tardar varios minutos (o más)... ten paciencia.
Fitting 5 folds for each of 3 candidates, totalling 15 fits
--- [lexicon_utils] Cargando recursos lingüísticos ---

--- ¡GridSearchCV (SVM) completado! ---
Tiempo total de búsqueda: 1.66 minutos


In [5]:
# --- 8. Guardar y Reportar Resultados ---

print("--- Resultados de la Fase 3 (SVM) ---")

try:
    # Extraer los mejores resultados del objeto grid_search
    best_score = grid_search.best_score_
    best_params = grid_search.best_params_

    print(f"Mejor F1-Macro (promedio de CV): {best_score:.4f}")
    print(f"Mejores Hiperparámetros encontrados: {best_params}")

    # Guardar los resultados en el archivo CSV que definimos en la Celda 2
    results_df = pd.DataFrame({
        "model": "LinearSVC (SVM)",
        "pipeline_base": "D (Negación) + FeatureUnion (Binary + Lexicons)",
        "best_avg_f1_macro": [best_score],
        "best_params": [str(best_params)] # Guardar como string
    })

    results_df.to_csv(RESULTS_PATH, index=False)
    print(f"\nResultados de la Fase 3 guardados en: {RESULTS_PATH}")

    # También muestra todos los resultados de la búsqueda
    print("\nDetalle de la búsqueda (GridSearchCV):")
    display(pd.DataFrame(grid_search.cv_results_))

except Exception as e:
    print(f"\n¡Error al guardar resultados! {e}")
    print("Verifica si el 'grid_search.fit()' de la celda anterior se completó correctamente.")

--- Resultados de la Fase 3 (SVM) ---
Mejor F1-Macro (promedio de CV): 0.4457
Mejores Hiperparámetros encontrados: {'classifier__C': 0.1}

Resultados de la Fase 3 guardados en: ..\results\fase_3_svm_results.csv

Detalle de la búsqueda (GridSearchCV):


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,11.113360,0.389653,0.813006,0.043105,0.1,{'classifier__C': 0.1},0.441937,0.440306,0.478936,0.422730,0.444540,0.445690,0.018313,1
1,40.194662,4.389018,0.667039,0.071004,1.0,{'classifier__C': 1},0.429158,0.418976,0.457384,0.402512,0.430572,0.427720,0.017900,2
2,59.730683,9.280642,0.426699,0.104072,10.0,{'classifier__C': 10},0.410583,0.412450,0.431974,0.393553,0.413986,0.412509,0.012199,3
